# ----------------------------------------
# 由于文件夹处理流程复杂且不一致；
# 所以单独将PIN分类列出
# ----------------------------------------

## 1. 读所有mag数据

In [343]:
import numpy as np
import os, pandas as pd

def get_Hyperparas():
    
    ''' 在此处最终定义几个超参数  '''
    # 定义备选的超参数
    mag_file = {'Huawei1':["Time", "3", "AcceX", "AcceY", "AcceZ", "5", "MagX", "MagY", "MagZ"],
               'Huawei2':["Time","MagX", "MagY", "MagZ" ],
               'OPPO':[ "Time", "3", "AcceX", "AcceY", "AcceZ", "4", "GyroX", "GyroY", "GyroZ", "5", "MagX", "MagY", "MagZ"],
               'Nexus':[ "Time", "3", "AcceX", "AcceY", "AcceZ", "4", "GyroX", "GyroY", "GyroZ", "5", "MagX", "MagY", "MagZ"]}
    base_folder_pin = 'D:/eclipse/workspace/TheGreatTrials/read/Huawei-1103-PIN/'
    base_folder_color = 'D:/eclipse/workspace/TheGreatTrials/read/Huawei-1103-Color/'
    base_folder_color_OPPO = 'D:/eclipse/workspace/TheGreatTrials/read/OPPO-1102-color/'
    base_folder_color_Nexus = 'D:/eclipse/workspace/TheGreatTrials/read/Nexus-1101-Color/'
    base_folder_Bg_NexusBgAPP_Double = 'D:/eclipse/workspace/TheGreatTrials/read/Nexus-1031-BgAPP-Double/'
    # 最终选择参数
    mag_columns = mag_file['Nexus']
    base_folder = base_folder_Bg_NexusBgAPP_Double # base_folder_color
    data_folders,examples = get_keywords(base_folder) # Get folders
    return mag_columns,base_folder,data_folders,examples

def get_keywords(base_folder):
    all_folders = []
    names_folder = []
    for folder in os.listdir(base_folder):
        if '.' not in folder and 'plot' not in folder: # 剔除文件夹下的readMe.txt和其他plot文件夹
            all_folders.append(folder) # 拿到所有文件夹
        if '_' in folder and folder[:-2] not in names_folder: #只以数字分隔
            names_folder.append(folder[:-2])
    print('All folders: \n',all_folders,'\nName folders:\n',names_folder)
    return all_folders, names_folder

# 读一个文件夹，并取出所有文件，并命名所有列
def read_a_folder(data_folder):
    # Read,Rename
    all_data = {}
    data_info = pd.read_csv(data_folder + 'magSensors.txt')
    #print( len(mag_columns) , len(data_info.iloc[0]) )
    assert len(mag_columns) == len(data_info.iloc[0]) # 文件读取列数和命名所需列数相等
    data_info.columns = mag_columns
    import copy
    ttt = copy.copy(mag_columns)
    ttt.remove('Time')
    ttt.remove('3')
    if '4' in mag_columns:
        ttt.remove('4')
        ttt.remove('5')
        data_info = data_info[ttt]
    return data_info

# 取得超参数
mag_columns,base_folder,data_folders,examples = get_Hyperparas()

# 合并每个文件夹里的文件
final_info = {}
for folder in data_folders:
    #print('%s' % folder)
    all_data = read_a_folder(base_folder + folder + '/')
    final_info[folder] = all_data
    
#如某类别含多个文件夹，就合并文件
use_mergeFiles = True
if use_mergeFiles:
    finally_you_win = {}
    for example in examples:
        if example + '_1' in final_info.keys():
            tttt1 = final_info[example + '_1']
            tttt2 = final_info[example + '_2']
            tttt3 = pd.concat([tttt1, tttt2], axis=0)
        else:
            tttt1 = final_info[example + '_1']
            tttt3 = tttt1
        finally_you_win[example] = tttt3
final_info = finally_you_win

All folders: 
 ['collectCalls_1', 'collectCalls_2', 'collectDownload_1', 'collectDownload_2', 'collectLight_1', 'collectLight_2', 'collectMusic_1', 'collectMusic_2', 'collectStatic_1', 'collectStatic_2', 'collectVideoRecord_1', 'collectVideoRecord_2'] 
Name folders:
 ['collectCalls', 'collectDownload', 'collectLight', 'collectMusic', 'collectStatic', 'collectVideoRecord']


## 2. 合并完毕，开始分析

In [344]:
# 本部分三个超参数： 1. 怎样切行   2. 怎样切列  3.怎样进行交叉验证
DevideParts = 4
sampele_rate = 2
rows_windowSize, rows_overlapSize = 200, 180
cols_usage = ['MagX', 'MagY','MagZ','MagX', 'MagZ']

all_division_files = []
# select divide
for i in range(DevideParts):
    label_number = 0
    # select file
    for keyword in final_info.keys():
        if 'recordAudio' not in keyword:
            # select rows
            one_category_data = final_info[keyword]
            # select cols
            #one_category_data = one_category_data['MagX']+one_category_data['MagY']+one_category_data['MagZ']
            one_category_data = one_category_data[cols_usage]
            # Resample
            locs = np.arange(0,len(one_category_data),sampele_rate)
            one_category_data = one_category_data.iloc[locs]
            cols = 1 if type(
                one_category_data) == pd.Series else one_category_data.shape[1]
            # select divide
            this_division_use_rows = len(one_category_data) // DevideParts
            one_category_part_data = one_category_data.iloc[
                i * this_division_use_rows:(i + 1) * this_division_use_rows]
            # select rows
            all_window_list_this_division = []
            window_no_overlap = rows_windowSize - rows_overlapSize
            count_windows = 0
            while window_no_overlap * count_windows + rows_windowSize < this_division_use_rows:
                window_data = one_category_part_data.iloc[window_no_overlap *
                                                          count_windows:
                                                          (window_no_overlap *
                                                           count_windows +
                                                           rows_windowSize)]
                #print(window_data.shape)
                all_window_list_this_division.extend(window_data.values)
                count_windows += 1
            # Generate one file all data
            all_window_array = np.array(all_window_list_this_division).\
                        reshape([-1,rows_windowSize*cols])
            labels = np.ones([count_windows, 1]) * label_number
            one_file_all = np.concatenate([all_window_array, labels], axis=1)
            print('Before conccat with labels, the file has shape:',
                  all_window_array.shape, ' After:', one_file_all.shape)
            label_number += 1
            # iter over files
            print('Processing file', keyword, 'It has shape',
                  one_file_all.shape, 'labeled with', labels[-1])
            all_division_files.extend(one_file_all)
    i += 1

# Finally concat data
from collections import Counter
all_division_files = np.array(all_division_files).reshape(
    [-1, 1 + rows_windowSize * cols])
print('Finally we combine data and got shape:', all_division_files.shape,
      'Labels:', Counter(all_division_files[:, -1]))

Before conccat with labels, the file has shape: (183, 1000)  After: (183, 1001)
Processing file collectLight It has shape (183, 1001) labeled with [ 0.]
Before conccat with labels, the file has shape: (180, 1000)  After: (180, 1001)
Processing file collectMusic It has shape (180, 1001) labeled with [ 1.]
Before conccat with labels, the file has shape: (163, 1000)  After: (163, 1001)
Processing file collectCalls It has shape (163, 1001) labeled with [ 2.]
Before conccat with labels, the file has shape: (42, 1000)  After: (42, 1001)
Processing file collectVideoRecord It has shape (42, 1001) labeled with [ 3.]
Before conccat with labels, the file has shape: (179, 1000)  After: (179, 1001)
Processing file collectStatic It has shape (179, 1001) labeled with [ 4.]
Before conccat with labels, the file has shape: (156, 1000)  After: (156, 1001)
Processing file collectDownload It has shape (156, 1001) labeled with [ 5.]
Before conccat with labels, the file has shape: (183, 1000)  After: (183, 1

## 3. 学习

In [ ]:
#all_division_files = all_division_files[np.where(all_division_files[:,-1] != 1)[0]]
#print(len(all_division_files))

feature = all_division_files[:, :-1]
label = all_division_files[:, -1]

# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=0.995)
pca.fit(feature)
#feature = pca.transform(feature)
print('Data shape after PCA:', feature.shape)

from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=DevideParts - 1)

cf1 = []
for train_index, test_index in tscv.split(feature):
#for tmd in range(1):

    # 划分
    from sklearn.model_selection import train_test_split
    #train_X,test_X, train_y, test_y = train_test_split(feature,label,test_size = 0.5,random_state = 0, shuffle=False)
    from collections import Counter
    train_X, test_X = feature[train_index], feature[test_index]
    train_y, test_y = label[train_index], label[test_index]

    #print('Train:', np.min(train_index), np.max(train_index), 'Test:',
    #      np.min(test_index), np.max(test_index))
    print(Counter(train_y), '\n', Counter(test_y))
    #print(train_index,test_index)

    # 分类
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=500,
                                 max_depth=50,
                                 random_state=0)
    clf.fit(train_X, train_y)
    
    # 分类
    from sklearn.neighbors import KNeighborsClassifier
    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(train_X, train_y)

    model = clf
    from sklearn.externals import joblib
    joblib.dump(clf, "RF_realTime.m")

    # 评价
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    predict_y = model.predict(test_X)
    confusion = confusion_matrix(test_y, predict_y)
    cf1.append(confusion)
    print(accuracy_score(test_y, predict_y))
    print('\n', confusion)

Data shape after PCA: (3612, 1000)
Counter({0.0: 183, 1.0: 180, 4.0: 179, 2.0: 163, 5.0: 156, 3.0: 42}) 
 Counter({0.0: 183, 1.0: 180, 4.0: 179, 2.0: 163, 5.0: 156, 3.0: 42})
0.919158361019

 [[183   0   0   0   0   0]
 [  0 167   0  12   1   0]
 [  0   0 163   0   0   0]
 [  0   0   0  24   1  17]
 [  0   0   0   0 179   0]
 [  0   0   0   0  42 114]]
Counter({0.0: 366, 1.0: 360, 4.0: 358, 2.0: 326, 5.0: 312, 3.0: 84}) 
 Counter({0.0: 183, 1.0: 180, 4.0: 179, 2.0: 163, 5.0: 156, 3.0: 42})


## 3. 混淆矩阵

In [1]:
import random
import numpy as np, pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
use_average_rows = 3
# Plot
normalize = True
cmap = plt.cm.Reds 
font_size = 22  # 15
algo_fontsize = 25  # 20
un_fig_size = (20, 8)
un_app_fig_size = (12, 7)
fig_size = (10,8)
confusion_fig_size = (10, 6)
window_fig_size = (15, 8)
precise_rate = "{:0.2f}"
def plot_action_cf(df_cm, title_, figure_saved, target_names, normalize, rotation,use_notation,font_size):
    import seaborn as sn
    import pandas as pd
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=fig_size)
    fig.set_tight_layout({'pad': 1})
    ax = sn.heatmap(ax=ax, data=df_cm, annot=use_notation, cmap=cmap, vmin=0, vmax=1,
                    fmt=".2f", annot_kws={"size": font_size - 15})
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=font_size - 15)  # colorbar
    ax.tick_params(labelsize=font_size - 15, width=1)
    ax.set_aspect(1)
    if target_names is not None:
        tick_marks = np.arange(0, len(target_names)) + 0.5 
        plt.xticks(tick_marks, target_names,fontsize=font_size-10, rotation=rotation) # [1,2,3,4,5,6,7,8,9,10,11]
        plt.yticks(tick_marks, target_names, rotation=0, fontsize=font_size-10)
    plt.savefig(base_folder + figure_saved + '.eps', format='eps')    
    plt.savefig(base_folder + figure_saved + '.pdf', format='pdf')
    plt.show()
    return

print(cf1[2])
cfion = np.array(cf1[1])
print('\n',cfion)
from jupyterthemes import jtplot
jtplot.reset() #系统默认选项
# 绘制
title_ = 'Classification of different Apps'
figure_saved = 'PIN分类结果'

# 转换混淆矩阵
final_confusion = []
arra1 = cf1[1]
sum_rows = np.sum(arra1,axis=1)
for i in range(len(arra1)):
    print(i,sum_rows[i])
    tmp = arra1[i] / sum_rows[i]
    final_confusion.append(tmp)
final_confusion = np.array(final_confusion)
cfion = np.round(final_confusion,3)

labels = list(final_info.keys())
labels = examples
plot_action_cf(cfion, title_, figure_saved, labels, normalize=False, rotation=45,use_notation=True,font_size = 30)

NameError: name 'cf1' is not defined